In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from sklearn.ensemble import RandomForestClassifier 

from xgboost import XGBClassifier

print(pd.__version__)

In [ ]:
dirc = '../../data/Santander-Customer-Transaction-Prediction/'

train = pd.read_csv(dirc + 'train.csv.zip', compression='zip')
test = pd.read_csv(dirc + 'test.csv.zip', compression='zip')
submission = pd.read_csv(dirc + 'sample_submission.csv.zip', compression='zip')

x = train.drop(['target', 'ID_code'], axis=1)
y = train['target'].astype('int')

test.drop('ID_code', axis=1, inplace=True)

train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.10) 

In [ ]:
model = XGBClassifier(n_estimators=5, n_jobs=-1)
model.fit(train_x, train_y, eval_set=[(val_x, val_y)], eval_metric='auc', early_stopping_rounds=5) 

pred = model.predict_proba(val_x)[:, 1]

print(roc_auc_score(val_y, pred))

In [ ]:
submission['target'] = model.predict_proba(test)[:, 1]
submission.to_csv('submission.csv', index=False)